In [1]:
import os
import glob
import tkinter as tk
import numpy as np
import open3d as o3d
from scipy.optimize import minimize
from scipy.spatial import cKDTree
 
# (Keep existing code to load strawberry model, orig_points, etc.)
 
orig_strawberry_points = []
# strawberry_model_pcd = 'strawberry_fruit_sampled.xyz'
strawberry_model_pcd = 'strawberry_fruit_sampled_no_leave.xyz'
with open(strawberry_model_pcd, 'r') as file:
    for line in file:
        x_str, y_str, z_str = line.strip().split()
        orig_strawberry_points.append((float(x_str), float(y_str), float(z_str)))
 
orig_strawberry_points = np.array(orig_strawberry_points)
print("orig_strawberry_points", orig_strawberry_points.shape)
 
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(orig_strawberry_points)
 
# Set model color to red and copy points
pcd.colors = o3d.utility.Vector3dVector(np.tile([1, 0, 0], (len(orig_strawberry_points), 1)))
orig_points = np.asarray(pcd.points).copy()
 
# Load an initial partial point cloud
# partial_pcd = o3d.io.read_point_cloud("segmented_pointcloud_separated/segmented_0000_0.ply")
# partial_pcd = o3d.io.read_point_cloud("front_orientation3_separated/segmented_0000_0.ply")
# partial_pcd = o3d.io.read_point_cloud("detection_data_654/filtered_points_0030_0.ply")
partial_pcd = o3d.io.read_point_cloud("8_April_2/filtered_points_0030_0.ply")
# partial_points = np.load("filtered_points2.npy")
# partial_pcd = o3d.geometry.PointCloud()
# partial_pcd.points = o3d.utility.Vector3dVector(partial_points)


partial_points = np.asarray(partial_pcd.points)
partial_pcd.colors = o3d.utility.Vector3dVector(np.tile([0, 0, 1], (partial_points.shape[0], 1)))
 
# --- Transformation: Align pcd points to the centroid of partial_pcd and rotate around x by 90 degrees ---
 
# Compute centroids of the strawberry model and the current partial point cloud.
strawberry_centroid = np.mean(orig_strawberry_points, axis=0)
partial_centroid = np.mean(partial_points, axis=0)
 
# Calculate translation needed to align the strawberry centroid to the partial centroid.
tx, ty, tz = partial_centroid - strawberry_centroid
 
# Define rotation angles in degrees; here, 90 degrees around the x-axis.
rx, ry, rz = 90, 0, 0
 
# Generate the rotation matrix for the given angles.
R = o3d.geometry.get_rotation_matrix_from_xyz((np.deg2rad(rx), np.deg2rad(ry), np.deg2rad(rz)))
 
# Apply the rotation followed by the translation to the strawberry points.
transformed_points = (R @ orig_strawberry_points.T).T + np.array([tx, ty, tz])
pcd.points = o3d.utility.Vector3dVector(transformed_points)
pcd.colors = o3d.utility.Vector3dVector(np.tile([1, 0, 0], (transformed_points.shape[0], 1)))
 
print("Applied translation (tx, ty, tz):", (tx, ty, tz))
print("Applied rotation (rx, ry, rz in degrees):", (rx, ry, rz))
 
 
coordinate_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.1, origin=[0, 0, 0])
 
pcd_display = o3d.geometry.PointCloud()
pcd_display.points = o3d.utility.Vector3dVector(transformed_points)
pcd_display.colors = o3d.utility.Vector3dVector(np.tile([1, 0, 0], (transformed_points.shape[0], 1)))
 
# Create a global visualizer (initial title will be used for the first partial)
vis = o3d.visualization.Visualizer()
# vis.create_window("Visualizer - segmented_0000_0", width=960, height=1080, left=2000, top=10)
vis.create_window("Visualizer - segmented_0030_0", width=1920, height=2160, left=2000)
vis.add_geometry(pcd_display)
vis.add_geometry(partial_pcd)
#vis.add_geometry(coordinate_frame)
 
transformation = np.eye(4)
 

# folder = "segmented_pointcloud_separated"
# folder = "front_orientation3_separated"
# folder = "detection_data_325"
# folder = "detection_data_539"
# folder = "detection_data_654"
folder = "8_April_2"

def apply_transformation():
    global transformation, pcd_display
    tx = slider_tx.get()
    ty = slider_ty.get()
    tz = slider_tz.get()
    rx = np.deg2rad(slider_rx.get())
    ry = np.deg2rad(slider_ry.get())
    rz = np.deg2rad(slider_rz.get())
    R = o3d.geometry.get_rotation_matrix_from_xyz((rx, ry, rz))
    T = np.eye(4)
    T[:3, :3] = R
    T[:3, 3] = [tx, ty, tz]
    transformation = T
    transformed_points = (R @ orig_points.T).T + np.array([tx, ty, tz])
    pcd_display.points = o3d.utility.Vector3dVector(transformed_points)
    pcd_display.colors = o3d.utility.Vector3dVector(np.tile([1, 0, 0], (transformed_points.shape[0], 1)))
    vis.update_geometry(pcd_display)
    print("Applied transformation:\n", T)
 
 
def move_to_centroid():
    # Compute initial guess.
    strawberry_centroid = np.mean(orig_points, axis=0)
    partial_centroid = np.mean(partial_points, axis=0)
    translation_init = partial_centroid - strawberry_centroid
    init_tx, init_ty, init_tz = translation_init
    init_rx, init_ry, init_rz = np.deg2rad(90), 0, 0
 
    
 
    # Define the objective function (Chamfer distance with a z-penalty).
    def chamfer_distance(params):
        tx, ty, tz, rx, ry, rz = params
        R = o3d.geometry.get_rotation_matrix_from_xyz((rx, ry, rz))
        transformed = (R @ orig_points.T).T + np.array([tx, ty, tz])
        tree_partial = cKDTree(partial_points)
        dists1, _ = tree_partial.query(transformed)
        tree_trans = cKDTree(transformed)
        dists2, _ = tree_trans.query(partial_points)
        chamfer = np.mean(dists1**2) + np.mean(dists2**2)
        penalty = 1000 * ((tz - init_tz)**2)  # Strongly penalize z shift
        return chamfer + penalty
    
    # Run optimization starting from the initial guess.
    x0 = [init_tx, init_ty, init_tz, init_rx, init_ry, init_rz]
    res = minimize(chamfer_distance, x0, method='Nelder-Mead', options={'maxiter': 100})
    optimal = res.x
    print("Optimization result:", optimal)
 
    slider_tx.set(optimal[0])
    slider_ty.set(optimal[1])
    slider_tz.set(optimal[2])
    slider_rx.set(np.rad2deg(optimal[3]))
    slider_ry.set(np.rad2deg(optimal[4]))
    slider_rz.set(np.rad2deg(optimal[5]))
 
    apply_transformation()
    print("Moved strawberry model to partial pointcloud's centroid.")
 
def save_transformation():
    current_filename=os.path.basename(partial_files[partial_index-1])
    identifier = current_filename.replace("segmented_", "").replace(".ply", "")
    
    # os.makedirs("transformation_matrix_", exist_ok=True)
    # np.save(f"transformation_matrix_/transformation_matrix_{identifier}.npy", transformation)
    # os.makedirs("transformation_orientation3_matrix", exist_ok=True)
    # np.save(f"transformation_orientation3_matrix/transformation_matrix_{identifier}.npy", transformation)
    # os.makedirs("transformation_detection_654_matrix", exist_ok=True)
    # np.save(f"transformation_detection_654_matrix/transformation_matrix_{identifier}.npy", transformation)
    os.makedirs("transformation_8_April_2_matrix", exist_ok=True)
    np.save(f"transformation_8_April_2_matrix/transformation_matrix_{identifier}.npy", transformation)

    transformed_points = (transformation[:3, :3] @ orig_points.T).T + transformation[:3, 3]
    pcd_save = o3d.geometry.PointCloud()
    pcd_save.points = o3d.utility.Vector3dVector(transformed_points)
    pcd_save.colors = o3d.utility.Vector3dVector(np.tile([1, 0, 0], (transformed_points.shape[0], 1)))
    # os.makedirs("transformation_strawberry_model_", exist_ok=True)
    # o3d.io.write_point_cloud(f"transformation_strawberry_model_/transformation_strawberry_model_{identifier}.ply", pcd_save)
    # os.makedirs("transformation_orientation3_model", exist_ok=True)
    # o3d.io.write_point_cloud(f"transformation_orientation3_model/transformation_strawberry_model_{identifier}.ply", pcd_save)
    # os.makedirs("transformation_detection_654_model", exist_ok=True)
    # o3d.io.write_point_cloud(f"transformation_detection_654_model/transformation_strawberry_model_{identifier}.ply", pcd_save)
    os.makedirs("transformation_8_April_2_model", exist_ok=True)
    o3d.io.write_point_cloud(f"transformation_8_April_2_model/transformation_strawberry_model_{identifier}.ply", pcd_save)

    print("Saved transformation matrix and model for", identifier)
 
# Process All function remains unchanged
def process_all():

    pattern = os.path.join(folder, "segmented_*.ply")
    files = sorted(glob.glob(pattern))
    for filepath in files:
        print("Processing", filepath)
        current_partial_pcd = o3d.io.read_point_cloud(filepath)
        current_points = np.asarray(current_partial_pcd.points)
        current_partial_pcd.colors = o3d.utility.Vector3dVector(np.tile([0, 0, 1], (current_points.shape[0], 1)))
        global partial_points, partial_pcd
        partial_points = current_points
        try:
            vis.remove_geometry(partial_pcd)
        except:
            pass
        partial_pcd = current_partial_pcd
        vis.add_geometry(partial_pcd)
        strawberry_centroid = np.mean(orig_points, axis=0)
        partial_centroid = np.mean(partial_points, axis=0)
        translation = partial_centroid - strawberry_centroid
        slider_tx.set(translation[0])
        slider_ty.set(translation[1])
        slider_tz.set(translation[2])
        slider_rx.set(90)
        slider_ry.set(0)
        slider_rz.set(0)
        apply_transformation()
       
        identifier = os.path.splitext(os.path.basename(filepath))[0]
        # identifier = os.path.basename(filename)
 
        save_transformation(identifier)
        vis.poll_events()
 
# Create a sorted list of partial pointcloud files and an index for going to the next partial.
# partial_files = sorted(glob.glob(os.path.join(folder, "segmented_*.ply")))
partial_files = sorted(glob.glob(os.path.join(folder, "filtered_points_*.ply")))
partial_index = 1   # Since the first file is already loaded
 
def next_partial():
    global partial_files, partial_index, partial_pcd, partial_points, vis
    if partial_index >= len(partial_files):
        print("No more partial pointclouds.")
        return
    filename = partial_files[partial_index]
    partial_index += 1
    print("Loading", filename)
    # Read the new partial
    new_partial = o3d.io.read_point_cloud(filename)
    new_partial_points = np.asarray(new_partial.points)
    new_partial.colors = o3d.utility.Vector3dVector(np.tile([0, 0, 1], (new_partial_points.shape[0], 1)))
    partial_points = new_partial_points
    try:
        vis.remove_geometry(partial_pcd)
    except:
        pass
    partial_pcd = new_partial
    # Update sliders so that the strawberry model is centered on the partial
    strawberry_centroid = np.mean(orig_points, axis=0)
    partial_centroid = np.mean(partial_points, axis=0)
    translation = partial_centroid - strawberry_centroid
    slider_tx.set(translation[0])
    slider_ty.set(translation[1])
    slider_tz.set(translation[2])
    slider_rx.set(90)
    slider_ry.set(0)
    slider_rz.set(0)
    apply_transformation()
    # Re-create the visualizer window with the new title (the title now shows the partial file name)
    new_title = f"Visualizer - {os.path.basename(filename)}"
 
    vis.destroy_window()
    vis.create_window(new_title, width=1920, height=2160, left=2000)
    vis.add_geometry(pcd_display)
    vis.add_geometry(partial_pcd)
    # vis.add_geometry(coordinate_frame)
    print("Updated visualizer with new title.")
 
def update_visualizer():
    vis.poll_events()
    vis.update_renderer()
    root.after(50, update_visualizer)
 
def create_slider_frame(parent, label_text, slider_from, slider_to, resolution):
    frame = tk.Frame(parent)
    frame.pack(pady=2)
    tk.Label(frame, text=label_text).pack(side=tk.LEFT)
    slider = tk.Scale(frame, from_=slider_from, to=slider_to, resolution=resolution,
                      orient=tk.HORIZONTAL, length=200, command=lambda x: apply_transformation())
    slider.pack(side=tk.LEFT)
    btn_minus = tk.Button(frame, text="-", command=lambda: slider.set(slider.get() - resolution))
    btn_minus.pack(side=tk.LEFT, padx=2)
    btn_plus = tk.Button(frame, text="+", command=lambda: slider.set(slider.get() + resolution))
    btn_plus.pack(side=tk.LEFT, padx=2)
    return slider
 
root = tk.Tk()
root.title("Strawberry Transformation")
 
slider_tx = create_slider_frame(root, "Translation X", -1, 1, 0.002)
slider_ty = create_slider_frame(root, "Translation Y", -1, 1, 0.002)
slider_tz = create_slider_frame(root, "Translation Z", -1, 1, 0.002)
slider_rx = create_slider_frame(root, "Rotation X (deg)", -180, 180, 0.5)
slider_ry = create_slider_frame(root, "Rotation Y (deg)", -180, 180, 0.5)
slider_rz = create_slider_frame(root, "Rotation Z (deg)", -180, 180, 0.5)
 
btn_move = tk.Button(root, text="Move to Centroid", command=move_to_centroid)
btn_move.pack(pady=5)
 
 
 
 
btn_save = tk.Button(root, text="Save Transformation", command=lambda: save_transformation())
btn_save.pack(pady=5)
btn_process_all = tk.Button(root, text="Process All Partials", command=process_all)
btn_process_all.pack(pady=5)
 
# New button to go to the next partial pointcloud
btn_next = tk.Button(root, text="Next Partial", command=next_partial)
btn_next.pack(pady=5)
 
root.after(50, update_visualizer)
root.mainloop()

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
orig_strawberry_points (893, 3)
[Open3D WARNING] Read PLY failed: unable to open file: 8_April_2\filtered_points_0030_0.ply
Applied translation (tx, ty, tz): (np.float64(nan), np.float64(nan), np.float64(nan))
Applied rotation (rx, ry, rz in degrees): (90, 0, 0)


c:\Users\Admin\anaconda3\envs\realsense\lib\site-packages\numpy\_core\fromnumeric.py:3596: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\Admin\anaconda3\envs\realsense\lib\site-packages\numpy\_core\_methods.py:130: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(


[Open3D WARNING] The number of points is 0 when creating axis-aligned bounding box.
Loading 8_April_2\filtered_points_0015_1.ply
Applied transformation:
 [[ 1.000000e+00  0.000000e+00  0.000000e+00  1.760000e-01]
 [ 0.000000e+00  6.123234e-17 -1.000000e+00 -4.000000e-03]
 [ 0.000000e+00  1.000000e+00  6.123234e-17  4.820000e-01]
 [ 0.000000e+00  0.000000e+00  0.000000e+00  1.000000e+00]]
Updated visualizer with new title.
Applied transformation:
 [[ 1.000000e+00  0.000000e+00  0.000000e+00  1.760000e-01]
 [ 0.000000e+00  6.123234e-17 -1.000000e+00 -4.000000e-03]
 [ 0.000000e+00  1.000000e+00  6.123234e-17  4.820000e-01]
 [ 0.000000e+00  0.000000e+00  0.000000e+00  1.000000e+00]]
Applied transformation:
 [[ 1.000000e+00  0.000000e+00  0.000000e+00  1.760000e-01]
 [ 0.000000e+00  6.123234e-17 -1.000000e+00 -4.000000e-03]
 [ 0.000000e+00  1.000000e+00  6.123234e-17  4.820000e-01]
 [ 0.000000e+00  0.000000e+00  0.000000e+00  1.000000e+00]]
Applied transformation:
 [[ 1.000000e+00  0.000000